In [5]:
import joblib
import time
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE

In [6]:
"""
准备数据集
"""
data_df1 = pd.read_csv(r"E:\Data\20230424-单人双次iFEMG肌力等级测试\normed_fea_bicps_br_absnew.csv", index_col = 0, header = [0, 1])
data_df2 = pd.read_csv(r"E:\Data\20230310-iFEMG肌力测试\normed_fea_bicps_br_absnew.csv", index_col = 0, header = [0, 1])

data_df = pd.concat([data_df1, data_df2], axis = 0, ignore_index = True)
# data_df = data_df2
print(data_df.shape)
data_df.head()

(452, 42)


bicps_br                                            tricps_br_medial  \
  subject_name height(cm) weight(kg) gender age label(kg)     subject_name   
0        lpy-1        182         82      1  21       0.0            lpy-1   
1        lpy-1        182         82      1  21       0.0            lpy-1   
2        lpy-1        182         82      1  21       0.0            lpy-1   
3        lpy-1        182         82      1  21       0.0            lpy-1   
4        lpy-1        182         82      1  21       0.0            lpy-1   

                                ...                            \
  height(cm) weight(kg) gender  ... mean_freq mean_power_freq   
0        182         82      1  ...  0.017160        0.000000   
1        182         82      1  ...  0.062069        0.036479   
2        182         82      1  ...  0.215211        0.319404   
3        182         82      1  ...  0.078377        0.101901   
4        182         82      1  ...  0.158286        0.205744   

  tricps_br_lateral                                                \
                FMG       mav       rms wave_length zero_crossing   
0          0.290186  0.639397  0.652967    0.447688      0.243478   
1          0.322176  0.262908  0.319657    0.456101      0.623982   
2          0.563522  0.390667  0.487624    0.526152      0.262519   
3          0.631699  0.426102  0.387842    0.759445      0.913563   
4          0.605924  0.177178  0.270645    0.345180      0.335128   

                                               
  slope_sign_change mean_freq mean_power_freq  
0          0.137429  0.000000        0.000000  
1          0.343830  0.393232        0.417870  
2          0.496322  0.771976        0.565790  
3          0.915901  0.687270        0.593396  
4          0.454993  1.000000        0.820658  

[5 rows x 42 columns]

In [7]:
"""
数据清洗 remove the data bellow:
    1. NaN value
    2. label is MVC
"""
# replace 'MVC' with NaN
data_df = data_df.replace('MVC', np.nan)
# delete NaN value
data_df = data_df.dropna(how = 'any')

print(data_df.shape)
data_df.dtypes

(452, 42)


bicps_br           subject_name          object
                   height(cm)             int64
                   weight(kg)             int64
                   gender                 int64
                   age                    int64
                   label(kg)            float64
tricps_br_medial   subject_name          object
                   height(cm)             int64
                   weight(kg)             int64
                   gender                 int64
                   age                    int64
                   label(kg)            float64
tricps_br_lateral  subject_name          object
                   height(cm)             int64
                   weight(kg)             int64
                   gender                 int64
                   age                    int64
                   label(kg)            float64
bicps_br           FMG                  float64
                   mav                  float64
                   rms                  

In [8]:
"""划分训练集和测试集"""
# 带FMG
fea_name_list = ['FMG', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']

x1_data_df = data_df["bicps_br"][fea_name_list]
x1_data_df.columns = [["bicps_br" for i in range(len(fea_name_list))], fea_name_list]
x2_data_df = data_df["tricps_br_medial"][fea_name_list]
x2_data_df.columns = [["tricps_br_medial" for i in range(len(fea_name_list))], fea_name_list]
x3_data_df = data_df["tricps_br_lateral"][fea_name_list]
x3_data_df.columns = [["tricps_br_lateral" for i in range(len(fea_name_list))], fea_name_list]
# 不带FMG
# x1_data = data_df["bicps_br"][['mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']].values
# x2_data = data_df["tricps_br_medial"][['mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']].values
# x3_data = data_df["tricps_br_lateral"][['mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']].values

y_data = data_df["bicps_br"]['label(kg)'].values
x_data_df = pd.concat([x1_data_df, x2_data_df, x3_data_df], axis = 1)
# x_data_df = np.concatenate((x1_data, x2_data, x3_data), axis=1)
print(f"dataset shape: {x_data_df.shape}")
print(f"label shape: {y_data.shape}")

dataset shape: (452, 24)
label shape: (452,)


In [9]:
"""
模型训练
"""
x_data = x_data_df.values
train_data_r, test_data_r, train_label, test_label = train_test_split(x_data, y_data, test_size = 0.2, random_state = 0)
# train_data = preprocessing.normalize(train_data_r, norm = 'max')
# test_data = preprocessing.normalize(test_data_r, norm = 'max')
scaler = preprocessing.StandardScaler().fit(train_data_r)
train_data = scaler.transform(train_data_r)
test_data = scaler.transform(test_data_r)


"""
Grid searching best parameters of SVR model
"""
# 记录开始训练时间
start_time = time.perf_counter()

# 自动选择合适的参数
svr = GridSearchCV(SVR(), param_grid={"kernel": ("linear", "rbf"), "C": np.logspace(-3, 3, 7), "gamma": np.logspace(-3, 3, 7)}, n_jobs=-1)
svr.fit(train_data, train_label)
print(f"Best params: {svr.best_params_}")

end_time = time.perf_counter()
print(f"training time(min): {(end_time - start_time)/60}")

score_test = svr.score(test_data, test_label)
score_train = svr.score(train_data, train_label)
print(f"{str(svr)} train score: {str(score_train)}")
print(f"{str(svr)} test score: {str(score_test)}")
test_pre = svr.predict(test_data)
train_pre = svr.predict(train_data)
print(f"train mean squared error: {mean_squared_error(train_label, train_pre)}")
print(f"test mean squared error: {mean_squared_error(test_label, test_pre)}")
    


Best params: {'C': 10.0, 'gamma': 0.01, 'kernel': 'rbf'}
training time(min): 3.2712933366666523
GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'kernel': ('linear', 'rbf')}) train score: 0.9466061520871283
GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'kernel': ('linear', 'rbf')}) test score: 0.8292950987036305
train mean squared error: 0.029765448783159534
test mean squared error: 0.0894546454746523
